In [9]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

In [15]:
image = Image.open('/Users/a2/Desktop/Bildschirmfoto 2024-11-04 um 21.15.37.png').convert("RGB")

In [3]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

THANK


In [1]:
import os
%pip install paddlepaddle

Note: you may need to restart the kernel to use updated packages.


In [12]:
from paddleocr import PaddleOCR, draw_ocr
import numpy as np


In [16]:
ocr = PaddleOCR(use_angle_cls=True, lang='en') 

[2024/11/04 21:15:59] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/a2/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/a2/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fa

In [25]:
image_array = np.array(image)

ocr_output_table = ocr.ocr(image_array)[0]

[2024/11/04 22:05:26] ppocr DEBUG: dt_boxes num : 100, elapsed : 0.31589293479919434
[2024/11/04 22:05:26] ppocr DEBUG: cls num  : 100, elapsed : 0.5760009288787842
[2024/11/04 22:05:38] ppocr DEBUG: rec_res num  : 100, elapsed : 11.309046983718872


In [31]:
import pandas as pd

# Helper functions to determine rows and columns based on Y and X coordinates.
def get_average_y(box):
    if len(box) < 4:
        raise ValueError("Box does not have enough coordinates to calculate average Y.")
    return (box[0][1] + box[2][1]) / 2

def get_average_x(box):
    if len(box) < 4:
        raise ValueError("Box does not have enough coordinates to calculate average X.")
    return (box[0][0] + box[2][0]) / 2

# Group by approximate rows based on y-coordinate proximity (tolerance of 10 pixels)
tolerance = 10
sorted_output_table = sorted(ocr_output_table, key=lambda item: get_average_y(item[0]))
rows_table = []
current_row_table = []
current_y_table = get_average_y(sorted_output_table[0][0])

for item in sorted_output_table:
    avg_y = get_average_y(item[0])
    if abs(avg_y - current_y_table) <= tolerance:
        current_row_table.append(item)
    else:
        # Sort current row by x-coordinate to align in columns
        rows_table.append(sorted(current_row_table, key=lambda item: get_average_x(item[0])))
        current_row_table = [item]
        current_y_table = avg_y

# Append the last row
if current_row_table:
    rows_table.append(sorted(current_row_table, key=lambda item: get_average_x(item[0])))

# Create a DataFrame from the structured rows and columns
aggressive_x_tolerance = 50  # Aggressive tolerance for creating new columns based on x-coordinate difference

table_data_2 = []
for row in rows_table:
    row_data = []
    previous_x = None
    for entry in row:
        current_x = get_average_x(entry[0])
        if previous_x is not None and (current_x - previous_x) > aggressive_x_tolerance:
            row_data.append("")  # Add empty cell for new column separation
        row_data.append(entry[1][0])
        previous_x = current_x
    table_data_2.append(row_data)

# Adjust for variable column lengths (fill missing columns with empty strings)
max_columns_2 = max(len(row) for row in table_data_2)
table_data_2 = [row + ["" ] * (max_columns_2 - len(row)) for row in table_data_2]

# Convert to DataFrame
df_2 = pd.DataFrame(table_data_2)



In [ ]:
df_2

In [23]:
import pandas
from img2table.document import Image

from img2table.ocr import EasyOCR

ocr = EasyOCR(lang=["en"])
src = '/Users/a2/Desktop/Bildschirmfoto 2024-11-07 um 18.03.27.png'
image = Image(src, 
              detect_rotation=False)
extracted_tables = image.extract_tables(ocr=ocr,
                                      implicit_rows=False,
                                      implicit_columns=False,
                                      borderless_tables=True,
                                      min_confidence=50)

In [27]:
extracted_tables[1].df

,0,1,2,3,4,5,6
0,Products,"69,958","67,184",None,"294,866","294,866","298,085"
1,Services,"24,972",22314,None,"96,169","96,169","85,200"
2,Total net sales (1),"94,930","89,498",None,"391,035","391,035","383,285"
3,Cost of sales:,None,None,None,None,None,None
4,Products,None,"42,586",None,"185,233","185,233","189,282"
5,Services,"6,485","6,485",None,"25,119","25,119","24,855"
6,Total cost of sales,"51,051","49,071",None,"210,352","210,352","214,137"
7,Gross margin,"43,879","40,427",None,"180,683","180,683","169,148"
8,Operating expenses:,None,None,None,None,None,None
9,Research and development,"7,765","7,307",None,"31,370","31,370","29,915"


In [1]:
import pandas